In [4]:

import numpy as np
from numba import njit
import edlib


def get_rc(s):
    map_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    l = []
    for c in s:
        l.append(map_dict[c])
    l = l[::-1]
    return ''.join(l)
def rc(s):
    map_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    l = []
    for c in s:
        l.append(map_dict[c])
    l = l[::-1]
    return ''.join(l)

def seq2hashtable_multi_test(refseq, testseq, kmersize=15, shift = 1):
    rc_testseq = get_rc(testseq)
    testseq_len = len(testseq)
    local_lookuptable = dict()
    skiphash = hash('N'*kmersize)
    for iloc in range(0, len(refseq) - kmersize + 1, 1):
        hashedkmer = hash(refseq[iloc:iloc+kmersize])
        if(skiphash == hashedkmer):
            continue
        if(hashedkmer in local_lookuptable):

            local_lookuptable[hashedkmer].append(iloc)
        else:
            local_lookuptable[hashedkmer] = [iloc]
    iloc = -1
    readend = testseq_len-kmersize+1
    one_mapinfo = []
    preiloc = 0
    while(True):
   
        iloc += shift
        if(iloc >= readend):
            break

        #if(hash(testseq[iloc: iloc + kmersize]) == hash(rc_testseq[-(iloc + kmersize): -iloc])):
            #continue
 
        hashedkmer = hash(testseq[iloc: iloc + kmersize])
        if(hashedkmer in local_lookuptable):

            for refloc in local_lookuptable[hashedkmer]:

                one_mapinfo.append((iloc, refloc, 1, kmersize))



        hashedkmer = hash(rc_testseq[-(iloc + kmersize): -iloc])
        if(hashedkmer in local_lookuptable):
            for refloc in local_lookuptable[hashedkmer]:
                one_mapinfo.append((iloc, refloc, -1, kmersize))
        preiloc = iloc

    

    return np.array(one_mapinfo)

def get_points(tuples_str):
    data = []
    num = 0
    for c in tuples_str:
        if(ord('0') <= c <= ord('9')):
            num = num * 10 + c - ord('0')
        elif(ord(',') == c):
            data.append(num)
            num = 0
    if(num != 0):
        data.append(num)
    return data

def calculate_distance(ref, query, ref_st, ref_en, query_st, query_en):
    A = ref[ref_st: ref_en]
    a = query[query_st: query_en]
    _a = rc(query[query_st: query_en])
    return min(edlib.align(A, a)['editDistance'], edlib.align(A, _a)['editDistance'])

def get_first(x):
    return x[0]


def calculate_value(tuples_str, ref, query):  

    slicepoints = np.array(get_points(tuples_str.encode()))
    if(len(slicepoints) > 0 and len(slicepoints) % 4 == 0):
        editdistance = 0
        aligned = 0
        preend = 0
        points = np.array(slicepoints).reshape((-1, 4)).tolist()
        points.sort(key=get_first)
        for onetuple in points:
            query_st, query_en, ref_st, ref_en = onetuple
            if(preend > query_st):
                return 0
            if(query_en - query_st < 30):
                continue
            preend = query_en
            if((calculate_distance(ref, query, ref_st, ref_en, query_st, query_en)/len(query[query_st:query_en])) > 0.1):
                continue
            editdistance += calculate_distance(ref, query, ref_st, ref_en, query_st, query_en)
            aligned += len(query[query_st:query_en])
        return max(aligned - editdistance, 0)
    else:
        return 0


In [5]:
#实验1
ref = '...'
query = '...'


In [6]:
data = seq2hashtable_multi_test(ref, query, kmersize=9, shift = 1)
data.shape

(63746, 4)

In [7]:
#实验2
ref = 'TGATTTAGAACGGACTTAGCAGACATTGAAACTCGAGGGGTATAGCAATAGATGCCCAAAAAGGTAAGCGCCATAAGCGTGGTTCTACGAGCCAGGTGCTCATGCCTAAGTTCTGCGCCTTCGCTGTCACTTGGAAATACTGTAATGGATCATGCCTAGGTTATGCGCCTTCGGGGTCACTTCAACATACTGTAATGGATCATGCCTAGGTTTTGCGTGTTCGCTGTCATTTCGAAATACTCCAATGGATGATGCCTAGGTTCTGTGCCTTCGCTGACGCATGGAAATACTTTAACGGATCATGCCCAGGCTCTGCGCCTTCGCTGAAACTTCGAAATACTCTAATGGATCATGCCTCGGTGCTCCACCTTCGCTTTCATTCCGAAATACTCTAATGGATCGCGTCCGTGTAACAACTTCGTACTGTTATATCGACCATCAGAATACCCATCCCTCGGGGAGGTAACCTATATTCACGTCGCAAGTTTCGATCTACAGTATGCTGACTGTTTGCCGCGATTTTAAGTCAAGAAGCGCGTCCACATGGTCGCGGTCGTCAACTTCAGTACGCTCATATGACACCAAAAGATCTACCTACAGCCCGTGCAGCTCGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCTCCCGCGCATCTCGACTTTTAAGCTCTATGGCACAACGTGTGGCGTTTGCCCCCGCGCAGCTCGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCCGTGCAGCCCGACTTTTGTACTCTAGTGCACGACGGGTGGCGTTTGCCCCCGCACCGCTCGACTTTTGTGATCTAGGGCACTACGAGTAGCGTTGGCCCAGACAGATCAACGCACATGGATTCTCTAACAGGCCCCGCGCTTCTCATTGGCCCGTGAGACGGGTCTGAGAGGAAGACATTAGGTAGATACGGAAAGCTTTGGCGTAGTTCGTATCTTTCAGCGGTGAAGCGTCTTCGGTCCGGGCTGCGTTATGCCTGCGGGAGGAAGGCTCCACTAGATGGTTTACGAGACATAATGTCAGCTTCCTAAAGGTACGGCAGCGCCTGCGTATATCACAGGACGAATTGTCAGTTTGCTAGGGGTACGGGAGCGCTTGCGTATTACATAGGACGAATCGTCAGCTTCCTAAAGGGACGGTAGCGCTTGCGTGTTACATAGGACGAATTGTCAGCTTCGTAAAGGTACGGTAGTTCTTGCGTATTACATAGGATGCATTGTCCGCTTCCTAAAGGTACGCTGGCGCTTGCGTATCACATAGGACGGATAGCGCGATTGCTAAAGGTACGGGAGCGCTTGCGTCTTAGAGCGCACGAATCGGATATAAGCTGCGCCCGCGTCTGGCGAGCAAAAATCGTGGGAGCCAGCGAGGGAAAAACTGCTCGGGCGACTTAAACGGAATTACAAGACTCATTGCCATCGAGGACGTTAGACTAAAGAGCCCCTGCGTGCCTCCTTTGTATAGCTCGATGTAGTGGCCCGTGTATGTGGAACAGGAATGCTCGATCTAAGGTAGTAGTGGCTACAGCTCCGAGAGTTTGCGTACTGCGGTGCCAGGGATTTTGCCTGCGGGTACAGCCTCTGCGCACGCCGGTCTGTGATCAAGAACTAAACTAGAGA'
query = 'TGATTTAGAACGGACTTAGCAGACATTGAAACTCGAGGGGTATAGCAATAGATGCCCAAAAAGGTAAGCGCCATAAGCGTGTTTCTACGAGCCAGGTGCTCATGCCTAAGTTCTGCGCCTTCGCTGTCACTGGGAAATACTGTAATGGATCATCCGTAGGTTATGCGCCTTCGGGGTCACTTCAACATACTGTAACGGATCGTGCCTAGGTTTTGCGTATTCGCTGTCATTTCGAATTACACCAATGGATGATGCCTAGGTTCTGTGCCTCCGCTGACGCATCGAAATACTTTAACGGATCGCGTCCGAGTAACAACTTCGTACTGTTATATAGGCAATCAGAATACCCATGCCTCGGGGAGGTAACCTATATTCACGTCGCAAGTTTCGATCTACAGTACTGTAGGTATATCTTTTGGTGTCATATGAGGGTACTGAACTTGACGACCGCGACCATGTGGATGCGCTTCTTGACTTAAAATCGCGGCAAACAGTAAGCATCCGTGAAGCTCGACTTTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCTCCCGCGCATCTCGAGTTGTAAGCTCTATGGCACAACGGGTGGCGTTTGCCGCCGAGCAGCTCGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGTGCTCTAGGGCACGACGGGTGGCCTTTGCCCCCGCGCAGCTGGACTTTTGTGCTCTAGGGCACGGCGGGTGGCGTTTGCCCTCCCCCAGCTTGACTACTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCCGCGCAGCTCGACTTTTGTGCTCTATGGCACGGGGGGTGGCGTTTGCCCTCGCCCAGCTTGACTTTTGCGCTCTAGGGCACGACGGGTGGCGTTTGCCGGCAAACGCCACACGTCGTGCCCTAGAGCACAAACGTCAAGCTGGGCGAGGGCAACCGCCACCCGCCCTGCCCTAGAGCACAAAAGTCGAGCTGCGCGGGCCCGCGCAGCTCGACTTTTGTGCTCTAGGACACGGCGGGTGGCGTTTGCCCTCGCCCAGCTTGACTCTTGTGCTCTAGGGCACGACGGGTGGCGTTTGCCCCAGCGCAGCCCGACTTTTGTACTCTAGAGCACGACGGTTGGCATTTGCCCCCGCACCGCTCGACTTTTGTGATCTAGGGCCCTAGGAGTAGCGTTGGCCAGCTTTCCGTATCTACCTAATGTCTTCCTCTCAGACCCGTCTCACGGGCCAATGAGAAGCGCGGGGCCTATTAGAGAATCCATGTGCGTTGATCTGTCTGCAGACAGCTCAACGCACATGGATTCGCTAGCAGGCCCCGCGCTTCTCATTGGCCCGTGAGACGGGTCTGAGAGGAAGACATAAGGTAGATACGGCAAGCTCACGTCCGTGTAACAACGTCGTACTGTTATATCGACCATCAGAATCCCCATCCCGCGAGGAGGTAACCTATATTCAGGTCGCAAGTTTCGATCTACAGTATTGGCGTAGTTCGTATCTTTCAGCGGTGAAGCTTCTTCGGTCCGGGCTGCGTTATGCCTGCTGGAGGACGGCTCCACTAGATGGTTTACGAGACATAATGTCCGCTTCCTAAAGGTACACTGGCGCTTGAGTATCACATAGGACGGATAGCTCGATTCCTAAAGGGACGGGAGCGCTTGCGTCTTAGAGCGCATGAATCGTCAGCTTCCCAAAGGGACCGTAGCGCTTGCGTGTTATATAGGAAGAATGGTCAGCTTTGTAAAGGTACGGTAGTTCTTGCGTATTACAGAGGATGCATTGTCTACTACCTAAAGGTACGGCAGCGCCTGCGTATATCACAGGACGAATTGTCAGTTTGCTAGGGGTACGGGAGCGCTTGCATATTACATAGGACGAATCGGATATAAGCTGCGCCCGCGTCTGGCGATAAAAAATCGTGGTAGCCAGCGAGGGAAAAACTGCTCGGGCGACTTAAACGGAATTAAAAGACTCATTGCCGTGACAGACTTCCGTATAGCAACCTCTGGGATGTCGATGCGGTGTCCCCAGTCTGCGCTGAGCGGGGGCAGACAGACTTAGTTATAGTATGCATCTGTTTTAGCTAGACATCACGACCTAGTGGGGTTCATGTTGAGATTCTAGGGCGGTACGCAGCCGGTGGATTATTACTTCCCCAGAAATTCTGACTTCGTCACTGGATGGATTGTACTATCCGGTCAACCTTACAAGGTTTCAACAGGGACGAAGGGTAAACGTATGAAGCTTGGATGCCGTTACCGTAAAGGGCCCTATTGAAGTGTCGAGGACGTTAGACTAAAGAGCCCCTGCGTGCCTCCTTTGTATAGCTCGAGGTAGTGGCCCGGATATGTGGAACAGGAATGCTCGATCTAAGGTAGTAGTGGGTACCGCTCCGAGAGTTTGCGTACTGCGGTGCCCGGGATTTTGCCTGCGGGTACAGCCTCTGCGCACGCCGGTCTGTAATCAAGAACTAAACTAGAGA'


In [8]:
data = seq2hashtable_multi_test(ref, query, kmersize=9, shift = 1)
data.shape

(2347, 4)

# DNA序列比对算法

本算法基于图的动态规划方法，实现DNA序列的最优比对。

In [ ]:
import heapq
import numpy as np
from collections import defaultdict

def reverse_complement(dna_seq):
    """计算DNA序列的反向互补序列"""
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join(complement.get(nuc, nuc) for nuc in reversed(dna_seq))

class Graph:
    """图数据结构，用于存储匹配关系"""
    def __init__(self):
        self.adjacency = {}
        
    def add_edge(self, src, dest, weight, meta):
        self.adjacency.setdefault(src, {})[dest] = {'weight': weight, 'meta': meta}
        
    def edge_exists(self, src, dest):
        return src in self.adjacency and dest in self.adjacency[src]
    
    def neighbors(self, src):
        if src not in self.adjacency:
            return []
        return list(self.adjacency[src].keys())
    
    def get_meta(self, src, dest):
        return self.adjacency[src][dest]['meta']
    
    def get_weight(self, src, dest):
        return self.adjacency[src][dest]['weight']

def search_occurrences_from_kmer_data(data, query_seq, ref_seq):
    """从kmer匹配数据中搜索出现模式"""
    occurrences = []
    
    # 对kmer匹配按query位置分组
    query_groups = defaultdict(list)
    for iloc, refloc, strand, kmersize in data:
        query_groups[iloc].append((refloc, strand, kmersize))
    
    # 寻找连续的kmer匹配来形成更长的匹配
    processed = set()
    
    for q_start in sorted(query_groups.keys()):
        if q_start in processed:
            continue
            
        matches = query_groups[q_start]
        
        for ref_start, strand, kmersize in matches:
            if (q_start, ref_start, strand) in processed:
                continue
                
            # 尝试扩展匹配
            current_q = q_start
            current_r = ref_start
            match_length = kmersize
            
            # 向右扩展
            next_q = current_q + kmersize
            next_r = current_r + kmersize if strand == 1 else current_r - kmersize
            
            while (next_q in query_groups and 
                   next_q < len(query_seq) and 
                   0 <= next_r < len(ref_seq)):
                
                found_extension = False
                for next_ref, next_strand, next_kmersize in query_groups[next_q]:
                    if next_strand == strand and abs(next_ref - next_r) <= 1:
                        match_length += next_kmersize
                        processed.add((next_q, next_ref, next_strand))
                        next_q += next_kmersize
                        next_r = next_ref + next_kmersize if strand == 1 else next_ref - next_kmersize
                        found_extension = True
                        break
                        
                if not found_extension:
                    break
            
            # 只保留长度足够的匹配
            if match_length >= 15:  # 至少15bp的匹配
                q_end = q_start + match_length
                r_end = ref_start + match_length if strand == 1 else ref_start - match_length
                
                occurrences.append({
                    'query_start': q_start,
                    'query_end': q_end,
                    'ref_start': ref_start,
                    'ref_end': r_end,
                    'length': match_length,
                    'strand': strand
                })
                
                processed.add((q_start, ref_start, strand))
    
    return occurrences

def build_graph_from_occurrences(occurrences, query_length):
    """从匹配构建图"""
    graph = Graph()
    
    for occ in occurrences:
        q_start = occ['query_start']
        q_end = occ['query_end']
        length = occ['length']
        
        graph.add_edge(q_start, q_end, 0, {
            'ref_start': occ['ref_start'],
            'ref_end': occ['ref_end'],
            'length': length,
            'strand': occ['strand']
        })
    
    return graph

def dynamic_programming_alignment(query_seq, ref_seq, graph):
    """使用动态规划找到最优比对路径"""
    q_len = len(query_seq)
    r_len = len(ref_seq)
    
    # 初始化
    heap = [(0, 0, 0, [])]  # (cost, query_pos, ref_pos, path)
    visited = set()
    best_alignments = []
    
    while heap:
        cost, q_pos, r_pos, path = heapq.heappop(heap)
        
        if (q_pos, r_pos) in visited:
            continue
        visited.add((q_pos, r_pos))
        
        # 到达终点
        if q_pos >= q_len:
            best_alignments.append({
                'cost': cost,
                'path': path,
                'alignments': extract_alignments_from_path(path)
            })
            continue
        
        # 探索可能的下一步
        next_steps = []
        
        # 1. 图中的边（匹配段）
        if q_pos in graph.adjacency:
            for q_dest in graph.neighbors(q_pos):
                meta = graph.get_meta(q_pos, q_dest)
                weight = graph.get_weight(q_pos, q_dest)
                
                new_path = path + [(q_pos, q_dest, meta['ref_start'], meta['ref_end'])]
                
                if meta['strand'] == 1:
                    new_r_pos = meta['ref_end']
                else:
                    new_r_pos = meta['ref_start']
                    
                next_steps.append((cost + weight, q_dest, new_r_pos, new_path))
        
        # 2. 单个碱基匹配/不匹配
        if q_pos < q_len and r_pos < r_len:
            match_cost = 0 if query_seq[q_pos] == ref_seq[r_pos] else 1
            next_steps.append((cost + match_cost, q_pos + 1, r_pos + 1, path))
        
        # 3. 插入/删除
        if q_pos < q_len:
            next_steps.append((cost + 1, q_pos + 1, r_pos, path))  # 插入
        if r_pos < r_len:
            next_steps.append((cost + 1, q_pos, r_pos + 1, path))  # 删除
        
        # 添加到堆中
        for next_step in next_steps:
            if (next_step[1], next_step[2]) not in visited:
                heapq.heappush(heap, next_step)
    
    return best_alignments[0] if best_alignments else None

def extract_alignments_from_path(path):
    """从路径中提取比对片段"""
    alignments = []
    for q_start, q_end, r_start, r_end in path:
        alignments.append((q_start, q_end, r_start, r_end))
    return alignments

def function(data):
    """主算法函数 - 返回格式为元组列表: [(query_start, query_end, ref_start, ref_end), ...]"""
    # 获取全局的query和ref序列（从上面的实验数据）
    global ref, query
    
    # 从kmer数据中搜索匹配
    occurrences = search_occurrences_from_kmer_data(data, query, ref)
    
    if not occurrences:
        return []
    
    # 构建图
    graph = build_graph_from_occurrences(occurrences, len(query))
    
    # 动态规划找最优路径
    result = dynamic_programming_alignment(query, ref, graph)
    
    if result and result['alignments']:
        return result['alignments']
    else:
        # 如果没有找到好的比对，返回最长的匹配
        if occurrences:
            # 按长度排序，返回最长的几个匹配
            occurrences.sort(key=lambda x: x['length'], reverse=True)
            alignments = []
            for occ in occurrences[:5]:  # 最多返回5个最长匹配
                alignments.append((
                    occ['query_start'],
                    occ['query_end'], 
                    occ['ref_start'],
                    occ['ref_end']
                ))
            return alignments
        return []

Result

In [10]:
# 调用算法函数
result = function(data)
print(f"找到 {len(result)} 个比对片段:")
for i, alignment in enumerate(result):
    q_st, q_en, r_st, r_en = alignment
    print(f"片段 {i+1}: Query[{q_st}:{q_en}] -> Ref[{r_st}:{r_en}], 长度: {q_en - q_st}")

# 转换为字符串格式用于评分
tuples_str = str(result)

找到 24 个比对片段:
片段 1: Query[309:327] -> Ref[409:427], 长度: 18
片段 2: Query[412:430] -> Ref[579:561], 长度: 18
片段 3: Query[440:458] -> Ref[551:533], 长度: 18
片段 4: Query[463:490] -> Ref[528:501], 长度: 27
片段 5: Query[507:561] -> Ref[607:661], 长度: 54
片段 6: Query[662:689] -> Ref[612:639], 长度: 27
片段 7: Query[712:748] -> Ref[712:748], 长度: 36
片段 8: Query[768:795] -> Ref[618:645], 长度: 27
片段 9: Query[795:822] -> Ref[695:722], 长度: 27
片段 10: Query[879:897] -> Ref[829:847], 长度: 18
片段 11: Query[915:933] -> Ref[776:758], 长度: 18
片段 12: Query[937:955] -> Ref[754:736], 长度: 18
片段 13: Query[972:999] -> Ref[719:692], 长度: 27
片段 14: Query[1000:1027] -> Ref[700:727], 长度: 27
片段 15: Query[1079:1097] -> Ref[829:847], 长度: 18
片段 16: Query[1105:1123] -> Ref[805:823], 长度: 18
片段 17: Query[1200:1263] -> Ref[991:928], 长度: 63
片段 18: Query[1272:1299] -> Ref[919:892], 长度: 27
片段 19: Query[1330:1375] -> Ref[930:975], 长度: 45
片段 20: Query[1418:1445] -> Ref[418:445], 长度: 27
片段 21: Query[1500:1527] -> Ref[1000:1027], 长度: 27
片段 22: Query

In [11]:
#Score
calculate_value(tuples_str, ref, query)

252